<a href="https://colab.research.google.com/github/qzh0004/NSF-IUSE/blob/main/notebooks/ISLP_Bias_Variance_Trade_off_CONV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Bias-Variance Trade-Off
================

The U-shape observed in test MSE is the result of two competing properties of ML methods.

The expected test MSE, for a given value $x_0$, can always be decomposed into the sum of three fundamental quantities: the variance of $\hat{f}(x_0)$, the squared bias of $\hat{f}(x_0)$ and the variance of the error terms $\epsilon$.

$\mathbb{E}(y_0-\hat{f}(x_0))^2=Var(\hat{f}(x_0)+[Bias(\hat{f}(x_0))]^2+Var(\epsilon)$

where

$Bias(\hat{f}(x_0))=f(x_0)-\mathbb{E}(\hat{f}(x_0))$

The purpose of this notebook is to illustrate the bias-variance trade-off using a numerical example.

Training data generation
------------------------

First we will write a function to generate a random sample. The data generation model is the following:

$y = 2sin(1.5x) + \epsilon$

with $\epsilon\sim\mathcal{N}(0,0.5)$

First we define the underlying functions: $f=2sin(1.5x)$, which is not known for real world problems, and the observed response $y=f+\epsilon$.

In [ ]:
import numpy as np
global sigma
sigma=0.5 # standard deviation of the error term (variance=sigma**2)
np.random.seed(seed=0) # define random seed so the results are repeatable.
def f(x):
    '''
    Returns a sample with instances/observations without noise (true but unknown value).
    '''
    yt = 2 * np.sin(x * 1.5)
    return yt

def y_m(x):
    '''
    Retrun a sample with instances/observations with noise/error (measured value).
    '''
    global sigma
    y = 2 * np.sin(x * 1.5) + np.random.normal(0, sigma, x.size)
    return y


1. The function ($f(x) = 2 \sin(1.5x)$) represents the true (but unknown) relationship between the input ($x$) and output ($y$). However, in real-world scenarios we typically do not know this true function (that's why we try to approximate or learn it from data).

2. The variable $y$ in the code is a noisy version of $f$. Specifically, $y$ adds a random noise/error term $\epsilon$ to the true function $f$. This noise represents measurement error, as well as other factors (e.g., instrument errors, missing variables, randomness in the process, etc.).

In practice, $f$ is usually unknown Besides measurement noise (i.e., unmeasurable variation), there are other factors that could contribute to $\epsilon$. For example, some variables that are useful in predicting $y$ may have not been included in $X$ because of our limited knowledge of the system or process for example. Since we don't measure them, $f$ cannot use them for its prediction.


Next we plot $f$ (underlying unknown function) as a line/curve, and a sample of $y$ (sample size 50, *i.e.*, 50 instances/observations) as dots.

In [ ]:
import matplotlib.pyplot as plt
n_obs = 50 # number of observations in each sample.
xm = np.linspace(0, 4.5, n_obs)
xt=np.linspace(0, 4.5, 100)
yt = f(xt)
plt.plot(xt, yt,'r')
ym = y_m(xm)
plt.plot(xm, ym, 'k.')
plt.show()

n_obs specifies the number of observations. Right now it's 50. You can change it to test out different values (e.g., 500 or 5000) to see how that change the plot (both the red line and the black points).

If n_obs is too small (e.g., 5), it would be difficult for us to imagine the true function form without knowing it.

 In practice, the noise or error magnitude can vary from application to application. It even depends on the units of the output. For example, $mm$ vs. $m$ used in length measurement. This magnitude is controlled in this simulated example? Can you simulate a scenario where the noise/error is very small?Change sigma=0.5 in the first code cell to sigma=0.05. Then click on Runtime menu and select Rull all.

Model fitting
=============

We will use least square regression (LSR) to fit a polynomial to the data. Actually, we will use multivariate linear regression, over a dataset built in the following way:

For each sample $x_{i}$ we build a vector $(1 , x_{i} , x_{i}^{2} , \dots , x_{i}^{n})$  and we use LSR to fit a function $g:\mathbb{R}^{n+1}\rightarrow\mathbb{R}$ to the training data. If you don't understand the following section, don't worry. It will not affect your progress of the sections followed.

One example is shown below where we fit the sample that we obtained earlier using an 8-th order polynomial. Still, the sample instances/points are shown as dots, the true model as red line/curve and the fitted/estimated model as blue line/curve.

In [ ]:
import matplotlib.pyplot as plt
from os import XATTR_SIZE_MAX
from sklearn.linear_model import LinearRegression  # liner regression model
from sklearn.preprocessing import PolynomialFeatures # polynommial features(extended features)

d=5; #degree of polynomial
poly_features = PolynomialFeatures(degree=d) # decide the maximal degree of the polynomial feature
xx=xm.reshape(-1, 1)
x_poly = poly_features.fit_transform(xx) # convert the original feature to polynomial feature
lin_reg = LinearRegression()
lin_reg.fit(x_poly,ym)
yp = lin_reg.predict(x_poly)
plt.plot(xm, ym, 'k.')
plt.plot(xt, yt,'r')
plt.plot(xm, yp,'b')


Even though polynomials might not be the most intuitive choice for a periodic function, for this particular range of $x$, a lot of polynmials provide good approximation.

We often pick certain models in practice due to convenience, simplicity of the model, familiarity with the methods, or the general approximation capability of the model (e.g., polynomials), even if the functional form is different. Remember that for most real applications, we don't have the knowledge of the true model, although sometimes we do know the form/structure of the model, e.g., sinusoidal, without knowing the specific parameter values.

Without thinking about bias-variance trade-off, which we will get to in a minute, we can observe that low degree (e.g., $d=1$ or $d=2$) fails to capture the sine wave shape (a.k.a., underfitting). while high degree (e.g., $d=16$) fits to the noise (a.k.a., overfitting).

If we repeat the experiment (with new random noise) for the same polynomial degree, we would not get exactly the same polynomial fit. This is clear from the next section where we repeat the model fitting process for different samples, which lead us to the concept of $Var(\hat{f}(x))$.

Model Bias and Variance
---------------

The following code generates a set of `n_sam` samples, allowing us to estimate $\hat{f}$ `n_sam` times. Those models are plotted as black lines.


In [ ]:
x0 = np.array([4.6]) # the test sample of x=4.6
xx_new=np.append(xx,x0)
xx_new=xx_new.reshape(-1, 1)
x0_poly = poly_features.fit_transform(xx_new) # convert the original feature to polynomial feature
degree = 3 # model polynomial order. you can change it to see how it affects the model
n_obs = 50 # number of observations in each sample.
n_sam= 20 # number of samples
yt0=np.zeros(n_sam) # true y, which is not known in the real world
ym0=np.zeros(n_sam) # measured y with error
yp0=np.zeros(n_sam) # predicted y
for i in range(n_sam):
    y = y_m(xm)
    model = LinearRegression()
    model.fit(x_poly,y)
    yp = model.predict(x_poly) # make predictions using trained Linear Regression model
    tmp=model.predict(x0_poly)
    yp0[i]=tmp[-1]
    ym0[i]=y_m(x0)
    yt0[i]=f(x0)
    plt.plot(xm, yp, 'k-')
plt.plot(xt, yt,'r')
plt.show()

The idea is to show how different random draws of noise affect the learned model, which highlights model variance when trained with different training sets. In real applications, we don't know the noise source(s) and how their realization, hence there's always uncertainty or variance in learned/trained models.



Even though $f(x)$ is fixed, the random noise in each sample leads the model to fit slightly differently, causing variation in predictions. Seeing multiple samples reveals the variability of the model, which is invisible when only looking at a single dataset.

For each sample, we get a different predicted value at $x=4.6$ (the unseen test sample), illustrated as the last points of all curves. (Note that this is also true for the training samples.) The spread (variance) of the 20 predictions at $x=4.6$ inform us about the reliability or stability of the model. A large spread indicates high variance; a small spread indicates more stable predictions.

# Calculation and visualization of bias and variance
Now we have predictions from 10 models trained by 10 samples. We can calucate the bias and variance and visualize them on graphs.

In [ ]:
import seaborn as sns
# Calculate stats
mean1, var1 = np.mean(yt0), np.var(yt0)
mean2, var2 = np.mean(yp0), np.var(yp0)
bin = [1.16-0.05, 1.16+0.05]  # Just 0.01 wide
# Plot overlapping histograms with KDE
plt.figure(figsize=(10, 5))
sns.histplot(yt0, bins=bin, kde=False, color='blue', label='True value', alpha=0.5)
sns.histplot(yp0, color='red', label='Predicted value', alpha=0.5, kde=True)

# Add vertical lines for the means
plt.axvline(mean1, color='blue', linestyle='--')
plt.axvline(mean2, color='red', linestyle='--')

# Annotate mean and variance near each vertical line
# (You can adjust the x and y positions to suit your data)
ymax = plt.ylim()[1]  # Get top limit of y-axis for positioning text
plt.text(mean1, 0.9*ymax, f"mean={mean1:.2f}\nvar={var1:.2f}", color='blue')
plt.text(mean2, 0.7*ymax, f"mean={mean2:.2f}\nvar={var2:.2f}", color='red')

plt.title("Overlapping Histograms with Mean & Variance")
plt.legend()
plt.show()

We plot both the true values (yt0) and predicted values (yp0) on the same histogram because we want to see how closely the distribution of predictions matches the (unknown in practice) true value distribution. Since we repeated the experiment with different random noise, we get multiple predicted values vs. a single “true” values.

The difference in location between the two histogram peaks (or the difference in mean) represent the bias.

yt0 is a single constant repeated 20 times. Because the true function is deterministic at a single $x$, it has no variation.

Typically, the true function $f(x)$ does not vary over multiple observations at the exact same $x$ —the underlying relationship is assumed deterministic at a specific $x$. Any variation in actual measurements (i.e., $y$) at the same $x$ would come from noise, not from $f$ itself.

## Bias
$Bias=|f(x_0)-\mathbb{E}[\hat{f}(x_0)]|$, where $f(x_0)$ is the true value and $\mathbb{E}[\hat{f}(x_0)]$ can be estimated as the mean or average of $\hat{f}(x_0)$.

In [ ]:
# Calculate bias (absolute difference of means)
Bias=abs(mean1-mean2)
print("Bias =", Bias)

The bias is not the average difference between predictions and their own mean, but rather the average difference between predictions and the true value. This is because the bias is about the systematic difference between the mean prediction and the true value.


## Variance


In [ ]:
# Calculate variance of the predictions around their own mean
Var=np.var(yp0-mean2) # Variance
print("Variance =", Var)

We measure the variance around the mean prediction rather than around the true value. Variance in the bias-variance decomposition focuses on how $\hat{f}$ fluctuates around its own mean (i.e., the expected prediction) across repeated samples. A high variance means small changes in training data can lead to large changes in predictions, making the model “unstable.”

Note that without subtracting mean from $\hat{f}(x_0)$ won't affect the variance calculation.

In [ ]:
Var=np.var(yp0) # Variance
print("Variance =", Var)

## Total error and mean squared error (MSE)
$\mathbb{E}[(y_0)-\hat{f}(x_0))^2]=Var(\hat{f}(x_0))+(Bias(\hat{f}(x_0)))^2+\mathbb{E}[\epsilon^2]$

Note that $\mathbb{E}[\epsilon^2]=Var(\epsilon)=\sigma^2$

In [ ]:
# Total error (theoretical formula for error decomposition: error = bias^2 + variance + noise_variance)
TotErr=Var+Bias**2+sigma**2
print("Total Error =", TotErr)
# MSE at x=4.6: mean of (prediction - true_value)^2 over all samples
MSE = np.mean((yp0 - yt0)**2)
print("MSE =", MSE)

Looking at $Var+Bias^2+\sigma^2$, each of these terms contribute to total error. Each term has a distinct role:
1. $Bias^2$ is the squared distance between the average prediction and the true value (systematic error).
2. $Var$ reflects how sensitive the model is to the particular training set (random fluctuations in fit).
3. $\sigma^2$ is the irreducible noise in the data.


Also note that $Var(\epsilon)$ is unknown in reality. But it can be estimated using measured response, i.e., $y$. If we assume $y$ follows (multivariate) Gaussion or Normal distribution, $Var(\epsilon)=Var(y)$.

In [ ]:
VarEst=np.var(ym0)
print(VarEst)

Note that because of the small sample size, which is 20 in this case, the estimated variance will not be an accurate estimate of the true value. Again, the true value is only known in simulated cases but unknown in any real applications.

## The Bias-Variance Trad-Off

Above we only calculated bias, variance and MSE for a fixed model flexibility (i.e., the polynomial order). Now we calculate them for a range of model flexibilities (i.e., a range of polynomial orders).

In [ ]:
import warnings
warnings.filterwarnings("ignore")

n_sam2 = 20   # number of samples
n_ord = 11    # maximum polynomial degree to examine

# Arrays now of length n_ord
bia_all  = np.zeros(n_ord)  # bias^2
var_all  = np.zeros(n_ord)  # variance
MSE_all1 = np.zeros(n_ord)  # MSE = bias^2 + variance + noise^2
MSE_all2 = np.zeros(n_ord)  # MSE from (ym0 - yp0)**2

for d in range(1, n_ord + 1):
    # Define polynomial features
    poly_features = PolynomialFeatures(degree=d)
    x_poly = poly_features.fit_transform(xx)
    x0_poly = poly_features.fit_transform(xx_new)

    # Arrays to store per-sample values
    yt0 = np.zeros(n_sam2)  # true y
    ym0 = np.zeros(n_sam2)  # measured y with noise
    yp0 = np.zeros(n_sam2)  # predicted y

    for i in range(n_sam2):
        ym = y_m(xm)
        model = LinearRegression()
        model.fit(x_poly, ym)

        # Predictions for training inputs and x0
        yp = model.predict(x_poly)
        tmp = model.predict(x0_poly)
        yp0[i] = tmp[-1]

        ym0[i] = y_m(x0)
        yt0[i] = f(x0)

    mean_true = np.mean(yt0)
    mean_pred = np.mean(yp0)

    # Compute bias^2
    bias_sq = (mean_true - mean_pred)**2
    bia_all[d-1] = bias_sq

    # Compute variance of predictions
    var_all[d-1] = np.var(yp0)

    # Theoretical MSE
    MSE_all1[d-1] = var_all[d-1] + bias_sq + sigma**2

    # Empirical MSE
    MSE_all2[d-1] = np.mean((ym0 - yp0)**2)

# Polynomial orders for the x-axis
degrees = range(1, n_ord + 1)

# Plot
plt.plot(degrees, bia_all, 'ro-', label='bias^2')
plt.plot(degrees, var_all, 'bo-', label='variance')
plt.plot(degrees, MSE_all1, 'go-', label='MSE1')
plt.plot(degrees, MSE_all2, 'yo-', label='MSE2')

plt.xlabel('Polynomial Order')
plt.legend()
plt.show()

Looking at the $bias^2$ (bia^2) and $Var$ (variance) curves as we increase the polynomial order from 1 to 11, we can see that bias typically decreases as the polynomial degree increases, because a more flexible model can better fit the underlying sine function. Variance typically increases with increasing polynomial degree, because the model becomes more sensitive to noise in each training sample.

MSE1 is theoretical, derived from $Var(\hat{f})+Bias^2+\sigma^2$. \\
MSE2 is empirical, the mean of squared differences between the predicted $\hat{f}(x_0)$ and the noisy measurement $y(x_0)$. \\
They both quantify errors, but MSE1 uses $\sigma^2$, the variance of the error term, which is unknown in real applications. MSE2 uses actual measurements without the knowledge of $\sigma$. Because of finite samples and randomness, the two curves generally align but may not match exactly. \\
Due to the reasons discussed above, in real applications, we can only obtain MSE2. However, as also discussed above, MSE2 represents MSE1 well in general.

The MSE curves often form a U-shape with respect to model complexity: if the model is too simple, bias is large; if too complex, variance is large. The optimal polynomial degree is usually in the middle, balancing bias and variance.

Low-degree polynomial (e.g., 1 or 2) is more rigid, hence its parameters are less affected by the randomness in training samples. This leads to small variance in models. However, the rigidness does lead to higher bias in general. \\
In contrast, higher-degree polynomials can adapt more closely to the underlying function, reducing systematic error (bias). However, the high flexibility also means that the model parameters are easier to be affected by (or overreact to) the randomness in the training samples, hence higher variance.

In practical settings, how do data scientists pick model complexity (e.g., polynomial degree, neural network size) when they can't measure true bias or variance directly? They typically rely on empirical error metrics (like MSE2), cross-validation, or hold-out validation sets. Concepts from the bias-variance trade-off guide them to avoid both underfitting and overfitting.

## References
This notebook is adapted and improved based on: https://gist.github.com/fabgoos/6788818